In [ ]:
# ---- 1. IMPORTS ----
import os, re, json, requests
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from datetime import datetime
from dotenv import load_dotenv
from time import sleep

# Load .env file and NLTK resources
load_dotenv()
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

# ---- 2. API KEYS ----
NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SLACK_WEBHOOK = os.getenv("SLACK_WEBHOOK")

# ---- 3. FETCH NEWS ----
def fetch_news():
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": "Artificial Intelligence",
        "pageSize": 100,
        "language": "en",
        "sortBy": "publishedAt",
        "apiKey": NEWSAPI_KEY
    }

    r = requests.get(url, params=params)
    try:
        data = r.json()
    except Exception:
        print("❌ Invalid JSON response from NewsAPI")
        print("Response text:", r.text)
        return []

    if data.get("status") != "ok":
        print("❌ NewsAPI error:", data.get("message", "Unknown error"))
        return []

    return data.get("articles", [])

# ---- 4. TEXT CLEANING ----
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    if not text:
        return ""
    text = re.sub(r"http\S+|www\S+", "", text)
    text = text.lower()
    return " ".join([w for w in text.split() if w not in STOPWORDS])

# ---- 5. GEMINI SENTIMENT (BATCH) ----
def get_sentiment_data_in_batch(headlines: list, batch_size=5) -> list:
    all_results = []

    for i in range(0, len(headlines), batch_size):
        batch = headlines[i:i + batch_size]
        numbered = "\n".join([f"{j+1}. {h}" for j, h in enumerate(batch)])

        prompt = f"""
You are a sentiment analysis expert.
Analyze each of the following news headlines independently.

Return a JSON array with each element like:
[{{"label": "Positive", "score": 0.85}}, ... ]

Headlines:
{numbered}
"""

        url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent?key={GEMINI_API_KEY}"
        headers = {"Content-Type": "application/json"}
        payload = {
            "contents": [{"role": "user", "parts": [{"text": prompt}]}],
            "generationConfig": {"temperature": 0.1, "maxOutputTokens": 512}
        }

        # Retry logic
        for attempt in range(3):
            try:
                r = requests.post(url, headers=headers, json=payload, timeout=90)
                r.raise_for_status()
                data = r.json()
                text_out = data["candidates"][0]["content"]["parts"][0]["text"]
                text_out = re.sub(r"```json|```", "", text_out).strip()
                parsed = json.loads(re.search(r"\[.*\]", text_out, re.DOTALL).group(0))

                for item in parsed:
                    label = item.get("label", "Neutral")
                    score = float(item.get("score", 0.0))
                    all_results.append((label, score))

                break  # success → stop retrying
            except Exception as e:
                if attempt < 2:
                    print(f"⚠ Timeout/error in batch {i//batch_size + 1}, retrying... ({attempt+1}/3)")
                    sleep(5)
                    continue
                else:
                    print(f"❌ Error in batch {i//batch_size + 1}: {e}")
                    all_results.extend([("Neutral", 0.0)] * len(batch))

        sleep(1)

    return all_results

# ---- 6. SLACK ALERT (with emojis + signed score) ----
def send_to_slack(title, url, sentiment, score):
    if not SLACK_WEBHOOK:
        return

    emoji_map = {
        "Positive": "✅",
        "Neutral": "⚪",
        "Negative": "❌"
    }
    emoji = emoji_map.get(sentiment, "")

    # Show + / - sign for score
    if sentiment == "Positive":
        score_text = f"+{score}"
    elif sentiment == "Negative":
        score_text = f"{score}"  # already negative
    else:
        score_text = "0"

    msg = f"{emoji} {sentiment.upper()} NEWS ALERT (Score: {score_text})\n• Title: {title}\n• Link: {url}"
    requests.post(SLACK_WEBHOOK, json={"text": msg})

# ---- 7. VISUALIZATIONS ----
def visualize(df):
    # --- Sentiment distribution ---
    plt.figure()
    df["sentiment_label"].value_counts().plot(kind="bar", color=["green","red","gray"])
    plt.title("Sentiment Distribution")
    plt.xlabel("Sentiment")
    plt.ylabel("Count")
    plt.savefig("sentiment_distribution.png")

    # --- Wordcloud ---
    text = " ".join(df["cleaned_content"])
    wc = WordCloud(width=800, height=400, background_color="white").generate(text)
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.savefig("wordcloud.png")

# ---- 8. MAIN PIPELINE ----
def run_pipeline():
    print("🔍 Fetching latest AI news...")
    articles = fetch_news()
    if not articles:
        print("No news fetched.")
        return

    print(f"📦 {len(articles)} articles fetched.")
    print("⚙️ Running sentiment analysis using Gemini 2.5 Flash Lite...")

    headlines = [f"{a.get('title','')} - {a.get('description','')}" for a in articles]
    sentiment_results = get_sentiment_data_in_batch(headlines, batch_size=5)

    results = []
    for i, a in enumerate(articles):
        title = a.get("title", "")
        desc = a.get("description", "")
        content = a.get("content", "")
        full_text = f"{title} {desc} {content}"
        cleaned = clean_text(full_text)

        label, score = sentiment_results[i]
        label_short = "Positive" if label.lower() == "positive" else "Negative" if label.lower() == "negative" else "Neutral"

        # Make score signed
        if label_short == "Negative":
            score = -abs(score)
        elif label_short == "Positive":
            score = abs(score)
        else:
            score = 0.0

        results.append({
            "src": a.get("source", {}).get("name"),
            "title": title,
            "description": desc,
            "url": a.get("url"),
            "publishedAt": a.get("publishedAt"),
            "content": content,
            "cleaned_content": cleaned,
            "sentiment_score": score,  # signed score
            "sentiment_label": label_short
        })

        send_to_slack(title, a.get("url"), label_short, score)

    df = pd.DataFrame(results)
    df.to_csv("ai_news_sentiment.csv", index=False)
    print("✅ Sentiment analysis complete.")
    print("📊 Generating visualizations...")
    visualize(df)
    print("🖼️ Visualizations saved: sentiment_distribution.png, wordcloud.png")

# ---- 9. RUN ----
if __name__ == "__main__":
    run_pipeline()
